In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm

In [2]:
MODEL_NAME = "BAAI/bge-base-en-v1.5"
BATCH_SIZE = 64
N_WORKERS = cpu_count()
SAVE_EVERY = 1_000  # Save checkpoint every N articles

In [3]:
news = pd.read_parquet("data/all_the_news_anon.pqt")
print(f"{len(news):,} articles")
news.head()

1,748,719 articles


,symbol,publishedDate,publisher,title,image,site,text,url,window_from,window_to,page,title_anon,text_anon
0,APLT,2025-01-28 17:30:00,Accesswire,Class Action Filed Against Applied Therapeutic...,https://images.financialmodelingprep.com/news/...,accessnewswire.com,"NEW YORK, NY / ACCESS Newswire / January 28, 2...",https://www.accessnewswire.com/newsroom/en/bus...,2025-01-22,2025-02-21,1,Class Action Filed Against __TARGET__ (__TARGE...,"NEW YORK, NY / __OTHER__ / January 28, 2025 / ..."
1,MSFT,2021-08-25 23:42:48,Benzinga,Apple Commits To Shoring Up Supply Chain Secur...,https://images.financialmodelingprep.com/news/...,benzinga.com,Apple Inc (NASDAQ: AAPL) is set to create a pr...,https://www.benzinga.com/news/21/08/22668742/a...,2021-08-11,2021-09-10,0,__OTHER__ Commits To Shoring Up Supply Chain S...,__OTHER__ Inc (__OTHER__: __OTHER__) is set to...
2,WDC,2025-10-23 11:05:51,Zacks Investment Research,Western Digital (WDC) Expected to Beat Earning...,https://images.financialmodelingprep.com/news/...,zacks.com,Western Digital (WDC) possesses the right comb...,https://www.zacks.com/stock/news/2775286/weste...,2025-10-19,2025-11-18,0,__TARGET__ (__TARGET__) Expected to __OTHER__ ...,__TARGET__ (__TARGET__) possesses the right co...
3,NFLX,2025-07-17 17:10:12,Bloomberg Markets and Finance,Netflix Reports Results | Closing Bell,https://images.financialmodelingprep.com/news/...,youtube.com,Comprehensive cross-platform coverage of the U...,https://www.youtube.com/watch?v=hvlxOAoLEJ4,2025-06-21,2025-07-21,0,__TARGET__ Reports Results | Closing Bell,Comprehensive cross-platform coverage of the U...
4,AMZN,2024-04-30 16:37:10,NYTimes,Amazon Reports $143.3 Billion in Revenue for F...,https://images.financialmodelingprep.com/news/...,nytimes.com,The company also reported that profit more tha...,https://www.nytimes.com/2024/04/30/technology/...,2024-04-27,2024-05-27,1,Amazon Reports $143.3 Billion in Revenue for F...,The company also reported that profit more tha...


In [4]:
# Check for and drop duplicate (url, symbol) pairs
n_dups = news.duplicated(subset=["url", "symbol"]).sum()
if n_dups > 0:
    print(f"Dropping {n_dups:,} duplicate (url, symbol) pairs")
    news = news.drop_duplicates(subset=["url", "symbol"], keep="first")

# Check for existing embeddings to skip
EMBEDDINGS_PATH = "data/news_embeddings.pqt"
try:
    existing = pd.read_parquet(EMBEDDINGS_PATH)
    already_done = set(zip(existing["url"], existing["symbol"]))
    print(f"Found {len(already_done):,} existing embeddings")
except FileNotFoundError:
    already_done = set()
    print("No existing embeddings found")

# Filter to only new articles
news["_key"] = list(zip(news["url"], news["symbol"]))
to_embed = news[~news["_key"].isin(already_done)].copy()
to_embed.drop(columns=["_key"], inplace=True)
news.drop(columns=["_key"], inplace=True)
print(f"{len(to_embed):,} articles to embed")

Dropping 2 duplicate (url, symbol) pairs
Found 9,998 existing embeddings
1,738,719 articles to embed


In [5]:
# Combine title + text for embedding
texts = (to_embed["title_anon"].fillna("") + " " + to_embed["text_anon"].fillna("")).str.strip().tolist()
urls = to_embed["url"].tolist()
symbols = to_embed["symbol"].tolist()
print(f"Avg length: {np.mean([len(t) for t in texts]):.0f} chars")

# Split into chunks for workers
def chunk_list(lst: list, n_chunks: int) -> list[list]:
    k, m = divmod(len(lst), n_chunks)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n_chunks)]

Avg length: 603 chars


In [6]:
# Worker function - each process loads its own model
def embed_chunk(args: tuple[int, list[str], str, int]) -> tuple[int, np.ndarray]:
    chunk_id, texts, model_name, batch_size = args
    
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(model_name)
    
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        normalize_embeddings=True,
    )
    return chunk_id, embeddings

In [7]:
def embed_batch_parallel(texts: list[str], urls: list[str], symbols: list[str]) -> pd.DataFrame:
    """Embed a batch of texts using multiprocessing and return a dataframe."""
    text_chunks = chunk_list(texts, N_WORKERS)
    args = [(i, tc, MODEL_NAME, BATCH_SIZE) for i, tc in enumerate(text_chunks)]
    
    with Pool(N_WORKERS) as pool:
        results = pool.map(embed_chunk, args)
    
    # Reassemble in order
    results.sort(key=lambda x: x[0])
    embeddings = np.vstack([r[1] for r in results])
    
    # Build dataframe
    emb_cols = [f"emb_{i}" for i in range(embeddings.shape[1])]
    df = pd.DataFrame(embeddings, columns=emb_cols)
    df["url"] = urls
    df["symbol"] = symbols
    return df[["url", "symbol"] + emb_cols]

In [ ]:
if len(texts) == 0:
    print("Nothing to embed")
else:
    n_total = len(texts)
    n_batches = (n_total + SAVE_EVERY - 1) // SAVE_EVERY
    
    # Load existing for appending
    try:
        existing_df = pd.read_parquet(EMBEDDINGS_PATH)
        all_dfs = [existing_df]
        print(f"Loaded {len(existing_df):,} existing embeddings")
    except FileNotFoundError:
        all_dfs = []
    
    print(f"Processing {n_total:,} articles in {n_batches} batches of ~{SAVE_EVERY:,}")
    print(f"Using {N_WORKERS} workers")
    print("-" * 60)
    
    for batch_idx in range(n_batches):
        batch_start = batch_idx * SAVE_EVERY
        batch_end = min(batch_start + SAVE_EVERY, n_total)
        
        batch_texts = texts[batch_start:batch_end]
        batch_urls = urls[batch_start:batch_end]
        batch_symbols = symbols[batch_start:batch_end]
        
        print(f"\nBatch {batch_idx + 1}/{n_batches}: articles {batch_start:,} - {batch_end:,} ({len(batch_texts):,} texts)")
        
        batch_df = embed_batch_parallel(batch_texts, batch_urls, batch_symbols)
        all_dfs.append(batch_df)
        
        # Save checkpoint
        combined = pd.concat(all_dfs, ignore_index=True)
        combined.to_parquet(EMBEDDINGS_PATH, index=False)
        
        done = batch_end
        pct = done / n_total * 100
        print(f"Progress: {done:,}/{n_total:,} ({pct:.1f}%) - checkpoint saved")
    
    print("-" * 60)
    print(f"Done! Total embeddings: {len(combined):,}")

Loaded 9,998 existing embeddings
Processing 1,738,719 articles in 1739 batches of ~1,000
Using 20 workers
------------------------------------------------------------

Batch 1/1739: articles 0 - 1,000 (1,000 texts)
Progress: 1,000/1,738,719 (0.1%) - checkpoint saved

Batch 2/1739: articles 1,000 - 2,000 (1,000 texts)
Progress: 2,000/1,738,719 (0.1%) - checkpoint saved

Batch 3/1739: articles 2,000 - 3,000 (1,000 texts)
Progress: 3,000/1,738,719 (0.2%) - checkpoint saved

Batch 4/1739: articles 3,000 - 4,000 (1,000 texts)
Progress: 4,000/1,738,719 (0.2%) - checkpoint saved

Batch 5/1739: articles 4,000 - 5,000 (1,000 texts)
Progress: 5,000/1,738,719 (0.3%) - checkpoint saved

Batch 6/1739: articles 5,000 - 6,000 (1,000 texts)
Progress: 6,000/1,738,719 (0.3%) - checkpoint saved

Batch 7/1739: articles 6,000 - 7,000 (1,000 texts)
Progress: 7,000/1,738,719 (0.4%) - checkpoint saved

Batch 8/1739: articles 7,000 - 8,000 (1,000 texts)
Progress: 8,000/1,738,719 (0.5%) - checkpoint saved

Batc

In [ ]:
# Verify final output
final_df = pd.read_parquet(EMBEDDINGS_PATH)
print(f"Final embeddings: {len(final_df):,} rows")
print(f"File size: {final_df.memory_usage(deep=True).sum() / 1e9:.2f} GB")
final_df.head()